# Repaso 6 – Pruebas de Hipótesis

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sanmartinfrancisco8/Estadistica_UGM_Notebooks/blob/main/Repaso_6_Prueba_Hipotesis.ipynb)

En este cuaderno repasaremos distintos tipos de pruebas estadísticas aplicadas a datos simulados.

Profundizaremos no solo en el cálculo numérico, sino también en la interpretación inferencial del proceso, relacionándolo con:

- Errores tipo I y II  
- Poder estadístico  
- Supuestos de cada prueba  
- Relevancia práctica vs. relevancia estadística  

---


## 1. Configuración inicial

Cargamos las librerías necesarias que usaremos en todas las pruebas.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
np.set_printoptions(precision=2)
pd.set_option('display.precision', 4)

## 2. Prueba Z para una media (σ conocida)

**Contexto:**
Una empresa quiere saber si el sueldo promedio actual de sus analistas supera los $50.000.  
Históricamente, la desviación estándar poblacional de los sueldos ha sido σ = 1.800.

- H₀: μ = 50.000  (el sueldo promedio **no ha cambiado**)  
- H₁: μ > 50.000  (el sueldo promedio **ha aumentado**)  
- Nivel de significancia: α = 0.05 (5%)

Usamos una **prueba Z unilateral (cola derecha)**.

In [ ]:
np.random.seed(123)
n = 120
sigma_poblacional = 1800
media_hipotetica = 50000

# Datos simulados de sueldos
sueldos = np.random.normal(loc=52000, scale=sigma_poblacional, size=n)

media_muestral = np.mean(sueldos)

z = (media_muestral - media_hipotetica) / (sigma_poblacional / np.sqrt(n))
p_valor = 1 - stats.norm.cdf(z)  # cola derecha

print(f"Media muestral: {media_muestral:,.2f}")
print(f"Estadístico Z: {z:.2f}")
print(f"p-valor (unilateral): {p_valor:.4f}")

### Interpretación

- Si `p-valor < α`, **rechazamos H₀** y concluimos que el sueldo promedio es significativamente mayor a 50.000.  
- Si `p-valor ≥ α`, **no rechazamos H₀**; los datos no entregan evidencia suficiente para afirmar que el sueldo ha aumentado.

En este ejemplo:

- Un p-valor muy pequeño indica un **bajo riesgo de cometer error Tipo I** si rechazamos H₀ (es decir, afirmar que aumentó el sueldo cuando en realidad no ha cambiado).

## 3. Prueba t de una media (σ desconocida)

Ahora supondremos que **no conocemos σ poblacional** y usamos la desviación estándar muestral.

- H₀: μ = 50.000  
- H₁: μ ≠ 50.000  (prueba bilateral)  
- Nivel de significancia: α = 0.05


In [ ]:
t_stat, p_valor_t = stats.ttest_1samp(sueldos, popmean=50000)

print(f"Estadístico t: {t_stat:.2f}")
print(f"p-valor (bilateral): {p_valor_t:.4f}")

### Interpretación

Como la prueba es **bilateral**, el p-valor representa la probabilidad de observar una media tan extrema (por encima o por debajo de 50.000) bajo el supuesto de que H₀ es cierta.

- Si `p-valor_t < 0.05`, concluimos que el sueldo promedio **es diferente** de 50.000.  
- De nuevo, un p-valor pequeño indica poca probabilidad de cometer **error Tipo I** si rechazamos H₀.


## 4. Prueba Chi-cuadrado de independencia

**Contexto:**
Marketing quiere saber si la **tasa de compra** depende del **tipo de producto**.

Se toma una muestra de clientes y se registra si compraron o no el producto A o B.

- H₀: *La decisión de compra es independiente del tipo de producto*.  
- H₁: *La decisión de compra depende del tipo de producto*.


In [ ]:
tabla = pd.DataFrame(
    {
        "Compra": [25, 15],
        "No Compra": [10, 30]
    },
    index=["Producto A", "Producto B"]
)

display(tabla)

chi2, p_valor_chi, gl, esperados = stats.chi2_contingency(tabla)

print(f"Estadístico Chi-cuadrado: {chi2:.2f}")
print(f"Grados de libertad: {gl}")
print(f"p-valor: {p_valor_chi:.4f}\n")
print("Tabla de frecuencias esperadas (si H0 fuera verdadera):")
display(pd.DataFrame(esperados, index=tabla.index, columns=tabla.columns))

### Interpretación

- Si `p-valor < 0.05`, rechazamos H₀ y concluimos que **existe asociación** entre tipo de producto y compra.  
- Si `p-valor ≥ 0.05`, los datos son compatibles con la independencia.

Las **frecuencias esperadas** permiten comparar visualmente cuánto se aleja la realidad del modelo bajo H₀.

## 5. ANOVA de un factor (comparación de medias entre ≥3 grupos)

**Contexto:**
Queremos comparar la **productividad promedio** de tres departamentos: Ventas, TI y RRHH.

- H₀: μ₁ = μ₂ = μ₃ (todas las medias son iguales).  
- H₁: Al menos una media es diferente.


In [ ]:
np.random.seed(2025)

n_por_grupo = 30

prod_ventas = np.random.normal(78, 8, n_por_grupo)
prod_ti     = np.random.normal(82, 7, n_por_grupo)
prod_rrhh   = np.random.normal(75, 6, n_por_grupo)

f_stat, p_valor_anova = stats.f_oneway(prod_ventas, prod_ti, prod_rrhh)

print(f"Estadístico F: {f_stat:.2f}")
print(f"p-valor ANOVA: {p_valor_anova:.4f}")

### Interpretación

- Si `p-valor ANOVA < 0.05`, concluimos que **al menos un departamento** tiene una productividad diferente.  
- El ANOVA no indica **cuál** grupo difiere; para eso se requieren pruebas post-hoc (Tukey, Bonferroni, etc.).

Aunque la diferencia pueda ser estadísticamente significativa, es importante evaluar si la diferencia en medias es **relevante en términos prácticos** (por ejemplo, en unidades de producción, ventas, etc.).

## 6. Resumen final

En este cuaderno vimos ejemplos aplicados de:

1. **Prueba Z** para una media con σ conocida.  
2. **Prueba t** para una media con σ desconocida.  
3. **Prueba Chi-cuadrado** de independencia en tablas de contingencia.  
4. **ANOVA de un factor** para comparar medias de tres o más grupos.

En todos los casos se enfatizó:

- Formulación de H₀ y H₁.  
- Cálculo del estadístico de prueba y p-valor.  
- Interpretación inferencial (decisión) y discusión de errores Tipo I/II y relevancia práctica.

Puedes modificar los parámetros de simulación (medias, desviaciones estándar, tamaños muestrales) para explorar cómo cambian los resultados y el poder estadístico de las pruebas.